In [2]:
from pyspark.sql import functions as f
from pyspark.sql import SparkSession
from pyspark.sql.types import (StructType, StructField, StringType,
                               IntegerType, BooleanType, TimestampType,
                              ArrayType, MapType, DateType)

from google.cloud import bigquery
from pyspark.sql.window import Window

spark = SparkSession \
        .builder.appName("DimTickerTypes") \
        .config("spark.jars.packages", "com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.24.0") \
        .getOrCreate()

#load types DF
types_df = spark \
        .read.option("recursiveFileLookup", "true") \
        .option("header", "true") \
        .parquet("gs://stocks-pipeline/raw-data/ticker_types")


print(types_df.show())

print(types_df.printSchema())
    

types_df = types_df.withColumn("TypeKey", f.row_number().over(Window.orderBy('code')))

types_df = types_df.withColumnRenamed("code","TypeCode") \
            .withColumnRenamed("description", "Description") \
            .withColumnRenamed("locale", "Locale")
            

types_df = types_df.fillna(value = 'Not Available', subset = ["Description"])
types_df= types_df.drop("asset_class")
# persist_df = types_df.persist()

# print(persist_df.show())
    
 # create a BigQuery client and dataset reference
client = bigquery.Client(project='noted-span-377814')
dataset_ref = client.dataset('Stocks_DW')

# create a BigQuery table and upload the data
table_ref = dataset_ref.table('DimTickerTypes')
job_config = bigquery.LoadJobConfig(write_disposition='WRITE_APPEND')
job = client.load_table_from_dataframe(types_df.toPandas(), table_ref, job_config=job_config)
spark.stop()


23/03/17 20:31:22 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/03/17 20:31:22 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/03/17 20:31:22 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/03/17 20:31:22 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator
23/03/17 20:31:36 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/17 20:31:36 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+-------+--------------------+------+
|asset_class|   code|         description|locale|
+-----------+-------+--------------------+------+
|     stocks|     CS|        Common Stock|    us|
|     stocks|    PFD|     Preferred Stock|    us|
|     stocks|WARRANT|             Warrant|    us|
|     stocks|  RIGHT|              Rights|    us|
|     stocks|   BOND|      Corporate Bond|    us|
|     stocks|    ETF|Exchange Traded Fund|    us|
|     stocks|    ETN|Exchange Traded Note|    us|
|     stocks|    ETV|Exchange Traded V...|    us|
|     stocks|     SP|  Structured Product|    us|
|     stocks|   ADRC|American Deposito...|    us|
|     stocks|   ADRP|American Deposito...|    us|
|     stocks|   ADRW|American Deposito...|    us|
|     stocks|   ADRR|American Deposito...|    us|
|     stocks|   FUND|                Fund|    us|
|     stocks| BASKET|              Basket|    us|
|     stocks|   UNIT|                Unit|    us|
|     stocks|     LT|   Liquidating Trust|    us|


23/03/17 20:31:36 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/17 20:31:36 WARN org.apache.spark.sql.execution.window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
